# SQL

https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

# Connect to MySQL

## Method 1 - mysql.connector

In [40]:
import mysql.connector
import getpass

In [41]:
cnx = mysql.connector.connect(
    host="localhost",
    user="root",
    password=getpass.getpass('Enter password: '), 
    database='sakila'
)

Enter password: ········


## Method 2 - pymysql

In [13]:
import pymysql as mysql
import pandas as pd
import getpass

In [20]:
cnx = mysql.connect(host='localhost',port=int(3306),user='root',passwd=getpass.getpass('Enter password: '),db='sakila')

Enter password: ········


# Both

In [24]:
def sql(query, params=None):
    cur = cnx.cursor()
    cur.execute(query)
    for x in cur:
        print(x)

In [25]:
def todf(query):
    df = pd.sql(query)
    return df

In [28]:
b = todf('SHOW DATABASES')
b

,Database
0,ads_507_supermarket
1,information_schema
2,module3
3,mysql
4,performance_schema
5,sakila
6,sys
7,world


## Create Database

In [ ]:
# execute queries
query = """
CREATE DATABASE IF NOT EXISTS ADS_507_Supermarket;
"""
sql(query)

In [ ]:
# Navigate to a db
cnx.select_db("db")

## Create Table

In [ ]:
create_order_table = """
    CREATE TABLE IF NOT EXISTS orders(
        Order_Id VARCHAR(100) NOT NULL,
        Company_Id VARCHAR(100) NOT NULL,
        Company_Name VARCHAR(255),
        Date DATE,
        Order_Value SMALLINT,
        Converted TINYINT UNSIGNED,
        PRIMARY KEY (Order_Id, Company_Id),
        INDEX (Company_Id),
        INDEX (Order_Id)
    )
;
"""

## Insert Data into Tables

In [ ]:
# Load orders datafraome to SQL table
for i, df_row in df.iterrows():
    query = """
    INSERT INTO df.orders VALUES (
        %s, %s, %s, %s, %s, %s)
        """
    params = tuple(df_row)
    sql(query, params)

## Views

In [ ]:
view_query = """
    CREATE VIEW company_metrics
        (company_name, year, month,
        meal_typ_sale, monthly_total,
        monthly_average, number_of_sales, year_to_date, yearly_total)
        AS
        SELECT o.Company_Name, YEAR(i.Date) AS Year, MONTHNAME(i.Date) AS Month,
            i.Type_of_Meal,
            SUM(i.Meal_Price) AS Monthly_Total,
            AVG(i.Meal_Price) AS Monthly_Average,
            COUNT(*) AS Number_of_Sales,
            SUM(SUM(i.Meal_Price))
                OVER (PARTITION BY o.Company_Name, Year(i.Date)
                        ORDER BY MONTH(STR_TO_DATE(month, '%M'))
                        ROWS UNBOUNDED PRECEDING) AS Year_to_Date,
            SUM(i.Meal_Price)
                OVER (PARTITION BY o.Company_Name, YEAR(i.Date)) AS Yearly_Total
        FROM orders AS o
            INNER JOIN invoice AS i
                ON o.Order_Id = i.Order_Id
        GROUP BY 1, 2, 3
        ORDER BY 1, 2, MONTH(STR_TO_DATE(month, '%M'))
"""

sql(view_query)

In [ ]:
pd.read_sql("SELECT * FROM company_metrics LIMIT 3;", cnx)

## Close Connection

In [ ]:
cnx.close()

## Method 3

In [ ]:
from sqlalchemy import create_engine

hostname="localhost"
dbname="module3"
uname="root"
pwd="Jd$0615!"


engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=uname, pw=pwd))
